# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas-docs.github.io/pandas-docs-travis/io.html#json
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas-docs.github.io/pandas-docs-travis/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
sample_json_listdict = json.load((open('data/world_bank_projects_less.json')))

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [8]:
# load data
json_listdict = json.load((open('data/world_bank_projects.json')))
json_df = pd.read_json('data/world_bank_projects.json')

### 1. Find the 10 countries with most projects

In [9]:
# verify country_namecode is the concat of countrycode and countryname column and no invalid entries 
# as group by count will exclude missing values
print(json_df.isnull().sum().loc[['countryname','countrycode','country_namecode']])
# ==> equal 0, no invalid entries
print(sum(json_df.apply(lambda x: x.loc['countryname']+'!$!'+x.loc['countrycode'] != x.loc['country_namecode'],axis=1)))
# ==> equal 0, verfiied
# check whether id is unique for project_name
print(json_df.loc[:,['id','project_name']].apply(lambda x: len(x.unique())==json_df.shape[0],axis=0))
# ==> true

countryname         0
countrycode         0
country_namecode    0
dtype: int64
0
id              True
project_name    True
dtype: bool


In [10]:
# groupby country_namecode, count id then and retrieve coutryname and coutrycode from country_namecod
country_count = json_df.loc[:,['id','country_namecode']].groupby('country_namecode').count().reset_index().rename(columns={'id':'count'})
country_count.head(5)

,country_namecode,count
0,Africa!$!3A,11
1,Antigua and Barbuda!$!AG,1
2,Arab Republic of Egypt!$!EG,2
3,Argentine Republic!$!AR,2
4,Bosnia and Herzegovina!$!BA,1


In [11]:
# get 10 countries with most projects
country_count.loc[:,'countryname'], country_count.loc[:,'countrycode'] = zip(*country_count.loc[:,'country_namecode'].apply(lambda x:x.split('!$!')))
country_count.sort_values('count', ascending=False).iloc[:10,:]

,country_namecode,count,countryname,countrycode
39,People's Republic of China!$!CN,19,People's Republic of China,CN
64,Republic of Indonesia!$!ID,19,Republic of Indonesia,ID
107,Socialist Republic of Vietnam!$!VN,17,Socialist Republic of Vietnam,VN
63,Republic of India!$!IN,16,Republic of India,IN
97,Republic of Yemen!$!RY,13,Republic of Yemen,RY
38,People's Republic of Bangladesh!$!BD,12,People's Republic of Bangladesh,BD
34,Nepal!$!NP,12,Nepal,NP
25,Kingdom of Morocco!$!MA,12,Kingdom of Morocco,MA
76,Republic of Mozambique!$!MZ,11,Republic of Mozambique,MZ
0,Africa!$!3A,11,Africa,3A


### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [12]:
# load data
json_listdict = json.load((open('data/world_bank_projects.json')))

In [13]:
# create dataframe containing elements in mjtheme_namecode
project_themes_sum_df = json_normalize(json_listdict,'mjtheme_namecode')

In [14]:
# ensure no missing values for code as name contain missing values which will be exclued by pandas group by and count, so use size instead
print(project_themes_sum_df.loc[:,'code'].apply(lambda x: x =='' or pd.isnull(x)).sum())

0


In [15]:
# find top 1 major themes
grouped = project_themes_sum_df.groupby('code')
top_10_themecode = grouped.size().sort_values(ascending=False).iloc[:10].index.values
def get_themename_from_themecode(code, grouped):
    i=-1
    group_data = grouped.get_group(code)
    while i < group_data.shape[0]:
        i +=1
        entry = group_data.loc[:,'name'].iloc[i]
        if entry and not pd.isnull(entry):
            break
    return group_data.loc[:,'name'].iloc[i]
top_10_themes = [get_themename_from_themecode(i,grouped) for i in top_10_themecode]
print(top_10_themes)

['Environment and natural resources management', 'Rural development', 'Human development', 'Public sector governance', 'Social protection and risk management', 'Financial and private sector development', 'Social dev/gender/inclusion', 'Trade and integration', 'Urban development', 'Economic management']


### 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [16]:
# create dictionary for filling missing reference
unique_code_key = project_themes_sum_df.loc[:,'code'].unique()
code_name_match_dict = {}
for code in unique_code_key:
    i = -1
    while i < project_themes_sum_df.shape[0]:
        i +=1
        entry_code = project_themes_sum_df.ix[i,'code']
        if entry_code == code:
            entry_name = project_themes_sum_df.ix[i,'name']
            if entry_name and not pd.isnull(entry_name):
                break
    code_name_match_dict[str(code)] = entry_name
print(code_name_match_dict)

{'8': 'Human development', '11': 'Environment and natural resources management', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [17]:
# fill the missing values in dataframe
print(project_themes_sum_df.head(10))
def fill_missing(x):
    if x.loc['name'] and not pd.isnull(x.loc['name']):
        return x.loc['name']
    else:
        return code_name_match_dict[str(x.loc['code'])]
project_themes_sum_df['name'] = project_themes_sum_df.apply(fill_missing, axis=1)
print(project_themes_sum_df.head(10))

  code                                          name
0    8                             Human development
1   11                                              
2    1                           Economic management
3    6         Social protection and risk management
4    5                         Trade and integration
5    2                      Public sector governance
6   11  Environment and natural resources management
7    6         Social protection and risk management
8    7                   Social dev/gender/inclusion
9    7                   Social dev/gender/inclusion
  code                                          name
0    8                             Human development
1   11  Environment and natural resources management
2    1                           Economic management
3    6         Social protection and risk management
4    5                         Trade and integration
5    2                      Public sector governance
6   11  Environment and natural resources mana